In [16]:
# I chose the Exponential Smoothing (Holt-Winters) model

# assignment1.py  —— 放在仓库根目录

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# 1) 用 RAW 链接读取（Actions 环境没有你的本地文件）
TRAIN_URL = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
TEST_URL  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"

# 2) 读数据并把时间设为小时频率索引
train = pd.read_csv(TRAIN_URL, parse_dates=["Timestamp"])
test  = pd.read_csv(TEST_URL,  parse_dates=["Timestamp"])

train = train.sort_values("Timestamp").set_index("Timestamp").asfreq("H")

# 3) 目标序列：转成数值并插值补齐
y_train = pd.to_numeric(train["trips"], errors="coerce").interpolate(limit_direction="both")

# 4) 建模：Holt-Winters（加性趋势+加性季节，24 小时季节性）
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated",
)

# 5) 拟合
modelFit = model.fit(optimized=True, use_brute=True)

# 6) 预测：和测试集等长（测试集有 744 行）
h = len(test)
forecast = modelFit.forecast(steps=h)

# 7) 按测试要求输出：一维数值 numpy 数组（长度 744）
pred = np.asarray(forecast, dtype=float).ravel()


/tmp/ipython-input-687737544.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train = train.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
